In [1]:
# notebook to compute log squared error of a mechanism compared to all data points from RCM experiments

In [2]:
import os
import sys
import cantera as ct
import numpy as np
import pandas as pd
import concurrent.futures
import rmgpy.chemkin
import subprocess

In [3]:
# load the mechanism

base_rmg = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/chem_annotated.cti'
improved_rmg = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/cutoff3_20230505_top50.cti'
aramco = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/aramco/AramcoMech3.0.cti'

In [4]:

# Take Reactor Conditions from Table 7 of supplementary info in
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
def run_simulation(T, P, X):
    # function to run a RCM simulation

    # gas is a global object
    t_end = 1.0  # time in seconds
    base_gas.TPX = T, P, X

    reactor = ct.IdealGasReactor(base_gas)
    reactor_net = ct.ReactorNet([reactor])

    times = [0]
    T = [reactor.T]
    P = [reactor.thermo.P]
    X = [reactor.thermo.X]  # mol fractions
    while reactor_net.time < t_end:
        try:
            reactor_net.step()
        except ct._cantera.CanteraError:
            print('Reactor failed to solve!')
            print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            return 0

        times.append(reactor_net.time)
        T.append(reactor.T)
        P.append(reactor.thermo.P)
        X.append(reactor.thermo.X)

    slopes = np.gradient(P, times)
    delay_i = np.argmax(slopes)
    return times[delay_i]

In [5]:
# Load the experimental conditions
ignition_delay_data = '/work/westgroup/harris.se/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
# ignition_delay_data = '/home/moon/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
df_exp = pd.read_csv(ignition_delay_data)
table_exp = df_exp[df_exp['Table'] < 13]
# Define Initial conditions using experimental data
tau_exp = table_exp['time (ms)'].values.astype(float)  # ignition delay
T7 = table_exp['T_C'].values  # Temperatures
P7 = table_exp['nominal pressure(atm)'].values * ct.one_atm  # pressures in atm
phi7 = table_exp['phi'].values  # equivalence ratios
# list of starting conditions
# Mixture compositions taken from table 2 of
# https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
concentrations = []
for i in range(0, len(phi7)):
    if phi7[i] == 0.3:
        x_diluent = 0.7821
        conc_dict = {
            'O2(2)': 0.2083,
            'butane(1)': 0.00962
        }
    elif phi7[i] == 0.5:
        x_diluent = 0.7771
        conc_dict = {
            'O2(2)': 0.2070,
            'butane(1)': 0.01595
        }
    elif phi7[i] == 1.0:
        x_diluent = 0.7649
        conc_dict = {
            'O2(2)': 0.2038,
            'butane(1)': 0.03135
        }
    elif phi7[i] == 2.0:
        x_diluent = 0.7416
        conc_dict = {
            'O2(2)': 0.1976,
            'butane(1)': 0.06079
        }
    else:
        raise ValueError
    if mech == 3:
        o2_conc = conc_dict.pop('O2(2)')
        conc_dict['O2'] = o2_conc

        butane_conc = conc_dict.pop('butane(1)')
        conc_dict['C4H10'] = butane_conc
    
    x_N2 = table_exp['%N2'].values[i] / 100.0 * x_diluent
    x_Ar = table_exp['%Ar'].values[i] / 100.0 * x_diluent
    x_CO2 = table_exp['%CO2'].values[i] / 100.0 * x_diluent
    conc_dict['N2'] = x_N2
    conc_dict['Ar'] = x_Ar
    if mech < 3:
        conc_dict['CO2(7)'] = x_CO2
    else:
        conc_dict['CO2'] = x_CO2
    concentrations.append(conc_dict)
assert len(T7) == len(concentrations)

In [ ]:
# # compute and save the delays
# print('Running Base RMG Delays')
# base_rmg_delays = np.zeros(len(concentrations))
# base_gas = ct.Solution(base_rmg)
# condition_indices = np.arange(0, len(concentrations))
# with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
#     for condition_index, delay_time in zip(condition_indices, executor.map(
#         run_simulation,
#         [T7[j] for j in condition_indices],
#         [P7[j] for j in condition_indices],
#         [concentrations[j] for j in condition_indices]
#     )):
#         base_rmg_delays[condition_index] = delay_time
# np.save('base_rmg_delays.npy', base_rmg_delays)

In [ ]:
# print('Running Improved RMG Delays')
# improved_rmg_delays = np.zeros(len(concentrations))
# base_gas = ct.Solution(improved_rmg)
# condition_indices = np.arange(0, len(concentrations))
# with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
#     for condition_index, delay_time in zip(condition_indices, executor.map(
#         run_simulation,
#         [T7[j] for j in condition_indices],
#         [P7[j] for j in condition_indices],
#         [concentrations[j] for j in condition_indices]
#     )):
#         improved_rmg_delays[condition_index] = delay_time
# np.save('improved_rmg_delays.npy', improved_rmg_delays)

In [ ]:
# print('Running Aramco Delays')
# aramco_delays = np.zeros(len(concentrations))
# improved_rmg_delays = np.zeros(len(concentrations))
# base_gas = ct.Solution(aramco)
# condition_indices = np.arange(0, len(concentrations))
# with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
#     for condition_index, delay_time in zip(condition_indices, executor.map(
#         run_simulation,
#         [T7[j] for j in condition_indices],
#         [P7[j] for j in condition_indices],
#         [concentrations[j] for j in condition_indices]
#     )):
#         aramco_delays[condition_index] = delay_time
# np.save('aramco_delays.npy', aramco_delays)

In [17]:
base_rmg_delays = np.load('base_rmg_delays_parallel.npy')
improved_rmg_delays = np.load('improved_rmg_delays_parallel.npy')
# aramco_delays = np.load('aramco_delays.npy')

In [18]:
tau_exp[tau_exp == 0] = np.nan

In [32]:
def calc_log_squared_error(mech_delays):
    return np.nansum(np.float_power(np.log(mech_delays) - np.log(tau_exp / 1000.0), 2.0))

In [33]:
calc_log_squared_error(base_rmg_delays)

95.58979196359489

In [34]:
calc_log_squared_error(improved_rmg_delays)

82.94925700798635

In [10]:
tau_exp

array([   nan,    nan,    nan,    nan,    nan,  63.4 ,   8.25,   8.25,
         3.36,   3.05,    nan,  54.5 ,  56.  ,  26.23,  26.23,  26.7 ,
        26.9 ,  42.  ,  44.1 ,  38.15,  39.34,  26.  ,  30.  ,  37.3 ,
        33.9 ,  10.07,   9.89,   2.45,   2.29, 128.  , 131.  ,  40.5 ,
        39.7 ,  13.6 ,  13.35,  10.5 ,  10.35,  12.8 ,  13.  ,  15.9 ,
        16.62,  10.3 ,  11.52,   4.18,   4.15, 300.  ,  69.  ,  68.  ,
       307.  , 204.  , 113.  , 115.  ,  18.  ,  19.07,   5.6 ,   5.59,
       221.  , 318.  ,  31.8 ,  32.  ,  14.86,  15.4 ,  16.6 ,  17.7 ,
        18.94,  24.6 ,  24.9 ,  28.  ,  27.1 ,  15.8 ,  15.3 ,   4.1 ,
         4.6 , 200.  , 139.  , 150.  ,  24.2 ,  21.2 ,  11.16,  10.4 ,
         6.5 ,   6.6 ,   7.26,   6.  ,   8.6 ,   3.68,   4.3 ,  86.5 ,
        73.9 ,  31.3 ,  33.  ,  30.9 ,  30.  ,  56.  ,  56.6 ,  67.9 ,
        66.4 ,  65.2 ,  67.1 ,  47.  ,  27.9 ,  28.05,  12.  ,  33.  ,
        32.8 ,  14.2 ,  11.9 ,   8.55,   7.7 ,   8.9 ,   8.55,  10.8 ,
      

In [7]:
base_rmg_delays

array([0.05828507, 0.09985099, 0.06010001, 0.038313  , 0.0271372 ,
       0.01025831, 0.00445635, 0.00417027, 0.00317239, 0.00307702,
       0.03292107, 0.01842591, 0.01842591, 0.01009465, 0.01009465,
       0.01561573, 0.01533243, 0.0250966 , 0.02530579, 0.01878335,
       0.01901561, 0.01291083, 0.01330269, 0.01074527, 0.01057943,
       0.00429174, 0.00422855, 0.00174308, 0.00176272, 0.02299398,
       0.02299398, 0.01288265, 0.01288265, 0.00524975, 0.00524975,
       0.00461754, 0.00461754, 0.00931406, 0.00931406, 0.00932967,
       0.00928417, 0.00557269, 0.00542476, 0.00237408, 0.00234083,
       0.04418948, 0.04678523, 0.04678523, 0.04764411, 0.022299  ,
       0.02310303, 0.02310303, 0.01107307, 0.01178512, 0.00511401,
       0.00505429, 0.05836062, 0.05661392, 0.01628771, 0.01628771,
       0.01118367, 0.01118367, 0.01152664, 0.01152664, 0.01978862,
       0.02565038, 0.02560348, 0.01924782, 0.01901561, 0.00890208,
       0.00862532, 0.00455598, 0.00455598, 0.04051895, 0.04181

In [38]:
table_exp[table_exp['Table'] == 1]['time (ms)']

0     0.00
1     0.00
2     0.00
3     0.00
4     0.00
5    63.40
6     8.25
7     8.25
8     3.36
9     3.05
Name: time (ms), dtype: float64

In [40]:
aramco_mech = ct.Solution(aramco)

In [45]:
aramco_mech.species_names.index('C4H10')

160